In [1]:
import pandas as pd
import numpy as np
import json
import csv
# import glob
# import requests
import seaborn as sns
# import matplotlib as plt
import urllib.request, json
from collections import OrderedDict, defaultdict

# from doltpy.cli import Dolt
# from doltpy.cli.read import read_pandas, read_pandas_sql

from pandas_read_xml import flatten, fully_flatten, auto_separate_tables
import pandas_read_xml as pdx

# import hospital xml

import xml.etree.ElementTree as ET

In [35]:
# create dictionary of xml names to column names
COLSDICT = {
    # should end up as npi instead?
    "name": "name",
    # procedures
    "charge_item_code": "code",
    "charge_item_descr": "long_description",
    # de-identified prices, look at how we did this with BSMH
    "charge_item_discountcashcharge": "self_pay",
    "charge_item_grosscharge": "gross",
    "charge_item_maxnegotiatedcharge": "max",
    "charge_item_minnegotiatedcharge": "min",
    # payer negotiated prices
    "charge_item_contracts_contract_charge": "price",
    "charge_item_contracts_contract_payer": "payer",
}

df = df_raw.copy()
# npi_number=1457321036
# add column for npi number
df["npi_number"] = npi_number


def col_clean(data):
    """clean columsn names of pd dataframe"""

    data.columns = (
        data.columns.str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("-", "_", regex=False)
        .str.replace("@", "", regex=False)
        .str.replace("|", "_", regex=False)
    )
    return data


# flatten the xml tree structure to get code, description, and standard charges
for i in range(5):
    # print(df.columns)
    df = df.pipe(flatten)
# filter for relevant codes
df = df[df["Charge|@Type"] == "HCPCS/CPT®"]
df = col_clean(df)
# drop if no code
df = df.dropna(subset=["charge_item_code"])

# define hospital name, for debugging
name = df.name.max()
lower_name = (
    df.name.str.lower()
    .str.replace(" ", "_", regex=False)
    .str.replace("-", "_", regex=False)
    .str.replace("@", "", regex=False)
    .str.replace("|", "_", regex=False)
    .max()
)

# remove commas from prices and make numeric
df[
    [
        "charge_item_discountcashcharge",
        "charge_item_grosscharge",
        "charge_item_maxnegotiatedcharge",
        "charge_item_minnegotiatedcharge",
    ]
] = (
    df[
        [
            "charge_item_discountcashcharge",
            "charge_item_grosscharge",
            "charge_item_maxnegotiatedcharge",
            "charge_item_minnegotiatedcharge",
        ]
    ]
    .replace({",": ""}, regex=False)
    .apply(pd.to_numeric, 1)
)
# rename columns
df.rename(columns=COLSDICT, inplace=True)

# increment goes before everything else
incremented = df["code"].copy()

counter = defaultdict(int)
for k, v in incremented.items():
    counter[v] += 1
    if counter[v] > 1:
        pre = v
        post = str("," + str(counter[v] - 1))
        incremented[k] = "%s%s" % (pre, post)
# replace code column with new incremented version
df["code"] = incremented


print("\n before procedures")
# print(df.head(1))

# create procedures table
procedures = df[["code", "npi_number", "long_description"]].copy()
procedures["short_description"] = np.nan
# set columns in right order
PROC_COLS=["code", "npi_number","short_description", "long_description"]

procedures=procedures[PROC_COLS]

# create procedures csv
new_filename = "procedures\\" + lower_name+"_" + str(npi_number) + ".csv"
procedures.to_csv(new_filename, index=False)

print("\n procedures")
# print(procedures.head(1))

print("DEBUG")
# CHARGES
# select only payer-non specific charge info
CHARGE_COLS = [
    "npi_number",
    "code",
    # "short_description",
    "self_pay",
    "gross",
    "max",
    "min",
]
# create df to convert deidentified stuff to prices table
charges = df[CHARGE_COLS].copy()
print("DEBUG 2")
# rename columns
charges.rename(columns=COLSDICT, inplace=True)

print("DEBUG 3")
# melt charges
def clean_melted_charges(dataframe):
    """used specifically to conver the metled non-payer specific charges to match prices schema"""

    # do the melting
    dataframe = dataframe.melt(id_vars=["npi_number", "code"], value_name="price")
    # set marker for inpatient & outpaitent prices
    dataframe["IP_OP"] = "UNSPECIFIED"
    dataframe["payer"] = "UNSPECIFIED"
    dataframe["caveat"] = np.nan
    dataframe["mode"] = np.NaN

    def replace_value(col, value_dict):
        for k, v in value_dict.items():
            dataframe.loc[dataframe.variable == k, col] = v

    # define melted variable name : variable new value
    payers = {
        "gross": "Gross Charge",
        "self_pay": "Discounted Cash Charge",
        "min": "De-identifiedMin",
        "max": "De-identifiedMax",
    }
    # replace payers with the above dict
    replace_value("payer", payers)

    # define melted variable name : variable new value
    modes = {
        "gross": "GROSS",
        "self_pay": "MINIMUM",
        "min": "MAXIMUM",
        "max": "EXPECTED",
    }
    # replace modes with the above dict
    replace_value("mode", modes)

    # return dataframe
    return dataframe[
        [
            "code",
            "npi_number",
            "payer",
            "IP_OP",
            "caveat",
            "mode",
            "price",
        ]
    ]


print("DEBUG 4")
# call function
new_charges = clean_melted_charges(charges)
print("DEBUG 5")
print("\n new charges")
# print(new_charges.head(1))

# END CHARGES
# # flatten the xml tree structure to get payer and charge out to each row
for i in range(2):
    # print(df.columns)
    df = df.pipe(flatten)
df = col_clean(df)
# rename columns
df.rename(columns=COLSDICT, inplace=True)
print("\n df columns before df price")
# print(df.head(1))

# remove commas from prices and make numeric
df["price"] = df["price"].replace({",": ""}, regex=False).apply(pd.to_numeric, 1)

# drop unnecessary columns
df.drop(
    columns=[
        # "short_description",
        "long_description",
        "charge_item_dept",
        "self_pay",
        "gross",
        "max",
        "min",
        "charge_item_note",
        "charge_item_qtytype",
        "charge_type",
        "createddate",
        "disclaimer",
    ],
    inplace=True,
)

# add nans as caveat
df["IP_OP"] = "UNSPECIFIED"
df["mode"] = "EXPECTED"
df["caveat"] = "NONE"

prices = pd.concat(
    [
        df[["code", "npi_number", "payer", "IP_OP", "caveat", "mode", "price"]],
        new_charges,
    ]
)
# create prices csv
new_filename = "prices\\"+lower_name+"_" + str(npi_number) + ".csv"
prices.to_csv(new_filename, index=False)

# create payers csv
new_filename = "payers\\" + lower_name+"_" + str(npi_number) + ".csv"
payers = pd.DataFrame(prices.payer.unique())
payers.to_csv(new_filename, index=False)

print(
    "\n succesful: "
    + name
    + " - "
    + str(npi_number)
    + " price nrows: "
    + str(len(prices))
)



 before procedures

 procedures
DEBUG
DEBUG 2
DEBUG 3
DEBUG 4
DEBUG 5

 new charges

 df columns before df price

 succesful: Bolivar Medical Center - 1457321036 price nrows: 158619


In [36]:
procedures.code.value_counts()
procedures.s.value_counts()

A9562    1
84146    1
J2997    1
27766    1
97530    1
        ..
V2787    1
71260    1
86765    1
V2632    1
50384    1
Name: code, Length: 1672, dtype: int64

In [65]:
# for row, blech in batch_1[['npi_number','url']]:
#     print(row)
#     print(blech)
zips=dict(zip(batch_1.npi_number.astype('str'),batch_1.url.str.replace(" ","_")))

In [66]:
zips

{'1063713659': 'https://www.cmhregional.com/sites/clinton/assets/uploads/27-3633811_CLINTON_MEMORIAL_HOSPITAL_StandardCharges_1.zip',
 '1801897038': 'https://www.conemaugh.org/Content/Uploads/Conemaugh%20Health%20System%20Redesign/files/611739000_Conemaugh_Health_System_Standard_Charges_12282020.zip',
 '1881726347': 'http://www.flemingcountyhospital.org/Content/Uploads/Fleming_County_Hospital/files/473937528_Fleming_County_Hospital_Standard_Charges_12202020.zip',
 '1780761866': 'https://www.havasuregional.com/Content/Uploads/Havasu_Regional/files/205220956%20Havasu%20Regional%20Medical%20Center%20Standard%20Charges%2012152020.zip',
 '1396760542': 'https://mylivingstonhospital.com/sites/livingston/assets/uploads/Livingston_Regional_Hospital_Standard_Charges_12192020.zip',
 '1831284280': 'https://www.yourlourdes.com/sites/lourdes/assets/uploads/36-4850536_LOURDES_HEALTH_StandardCharges.zip',
 '1386720183': 'http://www.martinsvillehospital.com/Content/Uploads/Sovah_Health_Rebuild/files/20

In [85]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

# filter out the npi number that keeps breaking
filtered= batch_1[batch_1.npi_number!='1700821808']

zips=dict(zip(filtered.npi_number.astype('str'),filtered.url.str.replace(" ","%20")))
# zips=dict(zip(batch_1.npi_number.astype('str'),batch_1.url))
extract_location = 'lifepoint_xmls'

def download_and_unzip(url, extract_to='.'):
    url = url
    # url = url.encode('unicode_escape')
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)
    file_str = str(path+"/"+str(zipfile.namelist()[0]))
    return file_str
    
new_zips={}
for k, v in zips.items():
    try:
        new_zips[k]=download_and_unzip(v, extract_location)
        print("yay! "+k)
    except Exception as e:
	    # print("ERROR : "+str(e))
        # new_zips[k]=np.nan
        next


yay! 1801897038.0


KeyboardInterrupt: 

In [78]:
new_zips

{'1063713659': '27-3633811_CLINTON_MEMORIAL_HOSPITAL_StandardCharges.xml',
 '1801897038': '611739000_Conemaugh_Health_System_Standard_Charges_12282020.xml',
 '1881726347': '473937528_Fleming_County_Hospital_Standard_Charges_12202020.xml',
 '1780761866': '205220956 Havasu Regional Medical Center Standard Charges 12152020.xml',
 '1396760542': '621762419_Livingston_Regional_Hospital_Standard_Charges_12192020.xml',
 '1831284280': '36-4850536_LOURDES_HEALTH_StandardCharges.xml',
 '1386720183': '202028539_Sovah_Health_Martinsville_Standard_Charges_12182020.xml',
 '1497708473': '472546387_Nason_Hospital_Standard_Charges_12202020.xml',
 '1477874337': '27-2451336_NORTH_ALABAMA_MEDICAL_CENTER_StandardCharges.xml',
 '1699096552': '27-2451336_SHOALS_HOSPITAL_StandardCharges.xml',
 '1326113762': '742791525 Palestine Regional Medical Center Standard Charges 12162020.xml',
 '1881977593': '452909143 Person Memorial Standard Charges 12152020.xml',
 '1922319037': '621762469_Riverview_Regional_Medical_Ce

In [2]:
xmls={'1063713659': '27-3633811_CLINTON_MEMORIAL_HOSPITAL_StandardCharges.xml',
 '1801897038': '611739000_Conemaugh_Health_System_Standard_Charges_12282020.xml',
 '1881726347': '473937528_Fleming_County_Hospital_Standard_Charges_12202020.xml',
 '1780761866': '205220956 Havasu Regional Medical Center Standard Charges 12152020.xml',
 '1396760542': '621762419_Livingston_Regional_Hospital_Standard_Charges_12192020.xml',
 '1831284280': '36-4850536_LOURDES_HEALTH_StandardCharges.xml',
 '1386720183': '202028539_Sovah_Health_Martinsville_Standard_Charges_12182020.xml',
 '1497708473': '472546387_Nason_Hospital_Standard_Charges_12202020.xml',
 '1477874337': '27-2451336_NORTH_ALABAMA_MEDICAL_CENTER_StandardCharges.xml',
 '1699096552': '27-2451336_SHOALS_HOSPITAL_StandardCharges.xml',
 '1326113762': '742791525 Palestine Regional Medical Center Standard Charges 12162020.xml',
 '1881977593': '452909143 Person Memorial Standard Charges 12152020.xml',
 '1922319037': '621762469_Riverview_Regional_Medical_Center_Standard_Charges_12182020.xml',
 '1245321181': '300811171_Rutherford_Regional_Medical_Center_Standard_Charges_12202020.xml',
 '1396842308': '621762468_Sage_West_Standard_Charges_12192020.xml',
 '1154419737': '202028539_Sovah_Health_Danville_Standard_Charges_12182020.xml',
 '1033160049': '62-1771866_SAINT_MARYS_REGIONAL_HEALTH_SYSTEM_StandardCharges.xml',
 '1134480155': '621866028_Athens_Woods_Starr_Regional_Standard_Charges_12192020.xml',
 '1467763458': '272618876_Trousdale_Medical_Center_Standard_Charges_12192020.xml',
 '1811086994': '320063628_Valley_View_Medical_Center_Standard_Charges_12192020.xml',
 '1336231232': '611275266 Western Plains Medical Complex Standard Charges 12152020.xml'}
path='lifepoint_xmls'
for k, v in xmls.items():
    xmls[k]=path+"/"+xmls[k]    

xmls

{'1063713659': 'lifepoint_xmls/27-3633811_CLINTON_MEMORIAL_HOSPITAL_StandardCharges.xml',
 '1801897038': 'lifepoint_xmls/611739000_Conemaugh_Health_System_Standard_Charges_12282020.xml',
 '1881726347': 'lifepoint_xmls/473937528_Fleming_County_Hospital_Standard_Charges_12202020.xml',
 '1780761866': 'lifepoint_xmls/205220956 Havasu Regional Medical Center Standard Charges 12152020.xml',
 '1396760542': 'lifepoint_xmls/621762419_Livingston_Regional_Hospital_Standard_Charges_12192020.xml',
 '1831284280': 'lifepoint_xmls/36-4850536_LOURDES_HEALTH_StandardCharges.xml',
 '1386720183': 'lifepoint_xmls/202028539_Sovah_Health_Martinsville_Standard_Charges_12182020.xml',
 '1497708473': 'lifepoint_xmls/472546387_Nason_Hospital_Standard_Charges_12202020.xml',
 '1477874337': 'lifepoint_xmls/27-2451336_NORTH_ALABAMA_MEDICAL_CENTER_StandardCharges.xml',
 '1699096552': 'lifepoint_xmls/27-2451336_SHOALS_HOSPITAL_StandardCharges.xml',
 '1326113762': 'lifepoint_xmls/742791525 Palestine Regional Medical Cen

# No function/for loops

In [54]:
for item in new_charges.columns.to_list():
    print("'"+str(item)+"',")

'npi_number',
'code',
'variable',
'price',
'IP_OP',
'payer',
'caveat',
'mode',


In [ ]:
for col in df.columns:
    print("'"+col+"':,")

# the big kahuna

## what's wrong with these ones

In [6]:
clinton=pdx.read_xml(
        pdx.read_xml_from_path(r"lifepoint_xmls\27-3633811_CLINTON_MEMORIAL_HOSPITAL_StandardCharges.xml", "utf8"),
        ["StandardCharges", "Facility"],
    )

def pipe_flat(df, n):
    for i in range(n):
        # print(df.columns)
        df = df.pipe(flatten)
    return df
        
clinton_flat=pipe_flat(clinton,6)
clinton_flat=col_clean(clinton_flat)
clinton_flat.columns
clinton_flat['charge_and_type']=clinton_flat['patient_type']+clinton_flat['patient_charge_type']
print(clinton_flat['charge_and_type'].value_counts())
print(clinton_flat['patient_charge_type'].value_counts())

clinton_flat=clinton_flat[clinton_flat.patient_charge_type=='HCPCS']


OutpatientHCPCS    1313
InpatientHCPCS      293
InpatientDRG        182
EmergencyHCPCS        6
Name: charge_and_type, dtype: int64
HCPCS    1612
DRG       182
Name: patient_charge_type, dtype: int64


,name,createddate,patient_type,patient_charge_type,patient_charge_item_code,patient_charge_item_description,patient_charge_item_qtytype,patient_charge_item_grosscharge,patient_charge_item_discountcashcharge,patient_charge_item_minnegotiatedcharge,patient_charge_item_maxnegotiatedcharge,patient_charge_item_contracts,charge_and_type
0,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,DRG,153,OTITIS MEDIA/URI WO MCC,Each,9837.490000,5902.494000,5369.490000,5369.490000,{'Contract': {'@Payer': 'COMMUNITY INSURANCE C...,InpatientDRG
1,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,DRG,175,PULM EMB W MCC OR AC COR P,Each,34021.950000,20413.170000,10762.470000,10762.470000,{'Contract': {'@Payer': 'COMMUNITY INSURANCE C...,InpatientDRG
2,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,DRG,176,PULMONARY EMB WO MCC,Each,9337.825000,5602.695000,5352.640000,9712.480000,{'Contract': [{'@Payer': 'MOUNT CARMEL INSURAN...,InpatientDRG
3,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,DRG,177,RESP INF/INFLAM W MCC,Each,20273.298888,12163.979333,1111.530000,22392.720000,"{'Contract': [{'@Payer': 'AETNA - (POS)', '@Ch...",InpatientDRG
4,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,DRG,178,RESP INF/INFLAM W CC,Each,17588.166666,10552.900000,6898.440000,10955.260000,"{'Contract': [{'@Payer': 'AETNA - (POS)', '@Ch...",InpatientDRG
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1789,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Outpatient,HCPCS,Q9967,"Locm 300-399mg/ml iodine,1ml",Each,33.553108,20.131865,0.101400,101.850000,"{'Contract': [{'@Payer': 'AETNA - (HMO)', '@Ch...",OutpatientHCPCS
1790,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Outpatient,HCPCS,S9123,Nursing care in home rn,Each,102.691071,61.614643,41.000000,41.000000,{'Contract': {'@Payer': 'UNITED HEALTHCARE INS...,OutpatientHCPCS
1791,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Outpatient,HCPCS,S9129,"Occupational therapy, in the",Each,324.000000,194.400000,97.000000,97.000000,{'Contract': {'@Payer': 'UNITED HEALTHCARE INS...,OutpatientHCPCS
1792,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Outpatient,HCPCS,S9131,Pt in the home per diem,Each,283.600000,170.160000,97.000000,97.000000,{'Contract': {'@Payer': 'UNITED HEALTHCARE INS...,OutpatientHCPCS


## failed zips, copy of for loop

In [ ]:
for npi_number, raw_xml in xmls.items():
    # count loops
    df_raw = pdx.read_xml(
            pdx.read_xml_from_path(raw_xml, "utf8"),
            ["StandardCharges", "Facility"],
        )
    try:
        
        
        

        # create dictionary of xml names to column names
        COLSDICT = {
            # should end up as npi instead?
            "name": "name",
            # procedures
            "charge_item_code": "code",
            "charge_item_descr": "long_description",
            # de-identified prices, look at how we did this with BSMH
            "charge_item_discountcashcharge": "self_pay",
            "charge_item_grosscharge": "gross",
            "charge_item_maxnegotiatedcharge": "max",
            "charge_item_minnegotiatedcharge": "min",
            # payer negotiated prices
            "charge_item_contracts_contract_charge": "price",
            "charge_item_contracts_contract_payer": "payer",
        }

        df = df_raw.copy()
        # npi_number=1457321036
        # add column for npi number
        df["npi_number"] = npi_number

        def col_clean(data):
            """clean columsn names of pd dataframe"""

            data.columns = (
                data.columns.str.lower()
                .str.replace(" ", "_", regex=False)
                .str.replace("-", "_", regex=False)
                .str.replace("@", "", regex=False)
                .str.replace("|", "_", regex=False)
            )
            return data

        # flatten the xml tree structure to get code, description, and standard charges
        
        for i in range(5):
            # print(df.columns)
            df = df.pipe(flatten)
        
        # clean columns    
        df = col_clean(df)
        
        # TODO maybe we've done this wrong?
        df = df[df["charge_type"] == "HCPCS/CPT®"]
        df = df.dropna(subset=["charge_item_code"])
        
        try:
            # filter for relevant codes
            successes += 1
        except:
            failed_zips[npi_number]=xmls[npi_number]
            print("Charge|@Type ERROR: "+ str(raw_xml))
            continue
        # drop if no code
        
        try:
            # filter for relevant codes
        except:
            print("charge_item_code ERROR: "+ str(raw_xml))
            continue

        # define hospital name, for debugging
        name = df.name.max()
        lower_name = (
            df.name.str.lower()
            .str.replace(" ", "_", regex=False)
            .str.replace("-", "_", regex=False)
            .str.replace("@", "", regex=False)
            .str.replace("|", "_", regex=False)
            .max()
        )

        # remove commas from prices and make numeric
        df[
            [
                "charge_item_discountcashcharge",
                "charge_item_grosscharge",
                "charge_item_maxnegotiatedcharge",
                "charge_item_minnegotiatedcharge",
            ]
        ] = (
            df[
                [
                    "charge_item_discountcashcharge",
                    "charge_item_grosscharge",
                    "charge_item_maxnegotiatedcharge",
                    "charge_item_minnegotiatedcharge",
                ]
            ]
            .replace({",": ""}, regex=False)
            .apply(pd.to_numeric, 1)
        )
        # rename columns
        df.rename(columns=COLSDICT, inplace=True)

        # increment goes before everything else
        incremented = df["code"].copy()

        counter = defaultdict(int)
        for k, v in incremented.items():
            counter[v] += 1
            if counter[v] > 1:
                pre = v
                post = str("," + str(counter[v] - 1))
                incremented[k] = "%s%s" % (pre, post)
        # replace code column with new incremented version
        df["code"] = incremented

        # print("\n before procedures")
        # print(df.head(1))

        # create procedures table
        procedures = df[["code", "npi_number", "long_description"]].copy()
        procedures["short_description"] = np.nan
        
        # clean
        
        # set columns in right order
        PROC_COLS = ["code", "npi_number", "short_description", "long_description"]

        procedures = procedures[PROC_COLS]

        # create procedures csv
        new_filename = "procedures\\" + lower_name + "_" + str(npi_number) + ".csv"
        procedures.to_csv(new_filename, index=False)

        # print("\n procedures")
        # print(procedures.head(1))

        # print("DEBUG")
        # CHARGES
        # select only payer-non specific charge info
        CHARGE_COLS = [
            "npi_number",
            "code",
            # "short_description",
            "self_pay",
            "gross",
            "max",
            "min",
        ]
        # create df to convert deidentified stuff to prices table
        charges = df[CHARGE_COLS].copy()
        # print("DEBUG 2")
        # rename columns
        charges.rename(columns=COLSDICT, inplace=True)

        # print("DEBUG 3")
        # melt charges
        def clean_melted_charges(dataframe):
            """used specifically to conver the metled non-payer specific charges to match prices schema"""

            # do the melting
            dataframe = dataframe.melt(id_vars=["npi_number", "code"], value_name="price")
            # set marker for inpatient & outpaitent prices
            dataframe["IP_OP"] = "UNSPECIFIED"
            dataframe["payer"] = "UNSPECIFIED"
            dataframe["caveat"] = "NONE"
            dataframe["mode"] = np.NaN

            def replace_value(col, value_dict):
                for k, v in value_dict.items():
                    dataframe.loc[dataframe.variable == k, col] = v

            # define melted variable name : variable new value
            payers = {
                "gross": "Gross Charge",
                "self_pay": "Discounted Cash Charge",
                "min": "De-identifiedMin",
                "max": "De-identifiedMax",
            }
            # replace payers with the above dict
            replace_value("payer", payers)

            # define melted variable name : variable new value
            modes = {
                "gross": "GROSS",
                "self_pay": "MINIMUM",
                "min": "MAXIMUM",
                "max": "EXPECTED",
            }
            # replace modes with the above dict
            replace_value("mode", modes)

            # return dataframe
            return dataframe[
                [
                    "code",
                    "npi_number",
                    "payer",
                    "IP_OP",
                    "caveat",
                    "mode",
                    "price",
                ]
            ]

        # print("DEBUG 4")
        # call function
        new_charges = clean_melted_charges(charges)
        # print("DEBUG 5")
        # print("\n new charges")
        # print(new_charges.head(1))

        # END CHARGES
        # # flatten the xml tree structure to get payer and charge out to each row
        for i in range(2):
            # print(df.columns)
            df = df.pipe(flatten)
        df = col_clean(df)
        # rename columns
        df.rename(columns=COLSDICT, inplace=True)
        # print("\n df columns before df price")
        # print(df.head(1))

        # remove commas from prices and make numeric
        df["price"] = df["price"].replace({",": ""}, regex=False).apply(pd.to_numeric, 1)

        # drop unnecessary columns
        df.drop(
            columns=[
                # "short_description",
                "long_description",
                "charge_item_dept",
                "self_pay",
                "gross",
                "max",
                "min",
                "charge_item_note",
                "charge_item_qtytype",
                "charge_type",
                "createddate",
                "disclaimer",
            ],
            inplace=True,
        )

        # add nans as caveat
        df["IP_OP"] = "UNSPECIFIED"
        df["mode"] = "EXPECTED"
        df["caveat"] = "NONE"

        prices = pd.concat(
            [
                df[["code", "npi_number", "payer", "IP_OP", "caveat", "mode", "price"]],
                new_charges,
            ]
        )
        # create prices csv
        new_filename = "prices\\" + lower_name + "_" + str(npi_number) + ".csv"
        prices.to_csv(new_filename, index=False)
        total_prices+=len(prices)

        # create payers csv
        new_filename = "payers\\" + lower_name + "_" + str(npi_number) + ".csv"
        payers = pd.DataFrame(prices.payer.unique())
        payers.to_csv(new_filename, index=False)

        print(
            "\n succesful: "
            + name
            + " - "
            + str(npi_number)
            + " price nrows: "
            + str(len(prices))
        )
    except:
        continue

# Make the first one a function

In [ ]:

def lp_imp1(npi_number,raw_xml):
    # count loops
    
    
    df_raw = pdx.read_xml(
        pdx.read_xml_from_path(raw_xml, "utf8"),
        ["StandardCharges", "Facility"],
    )

    # create dictionary of xml names to column names
    COLSDICT = {
        # should end up as npi instead?
        "name": "name",
        # procedures
        "charge_item_code": "code",
        "charge_item_descr": "long_description",
        # de-identified prices, look at how we did this with BSMH
        "charge_item_discountcashcharge": "self_pay",
        "charge_item_grosscharge": "gross",
        "charge_item_maxnegotiatedcharge": "max",
        "charge_item_minnegotiatedcharge": "min",
        # payer negotiated prices
        "charge_item_contracts_contract_charge": "price",
        "charge_item_contracts_contract_payer": "payer",
    }

    df = df_raw.copy()
    # npi_number=1457321036
    # add column for npi number
    df["npi_number"] = npi_number

    def col_clean(data):
        """clean columsn names of pd dataframe"""

        data.columns = (
            data.columns.str.lower()
            .str.replace(" ", "_", regex=False)
            .str.replace("-", "_", regex=False)
            .str.replace("@", "", regex=False)
            .str.replace("|", "_", regex=False)
        )
        return data

    # flatten the xml tree structure to get code, description, and standard charges
    
    for i in range(5):
        # print(df.columns)
        df = df.pipe(flatten)
    
    # clean columns    
    df = col_clean(df)
    
    # TODO maybe we've done this wrong?
    try:
        # filter for relevant codes
        df = df[df["charge_type"] == "HCPCS/CPT®"]
    except:
        print("Charge|@Type ERROR: "+ str(raw_xml))
        continue
    # drop if no code
    
    try:
        # filter for relevant codes
        df = df.dropna(subset=["charge_item_code"])
    except:
        print("charge_item_code ERROR: "+ str(raw_xml))
        continue

    # define hospital name, for debugging
    name = df.name.max()
    lower_name = (
        df.name.str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("-", "_", regex=False)
        .str.replace("@", "", regex=False)
        .str.replace("|", "_", regex=False)
        .max()
    )

    # remove commas from prices and make numeric
    df[
        [
            "charge_item_discountcashcharge",
            "charge_item_grosscharge",
            "charge_item_maxnegotiatedcharge",
            "charge_item_minnegotiatedcharge",
        ]
    ] = (
        df[
            [
                "charge_item_discountcashcharge",
                "charge_item_grosscharge",
                "charge_item_maxnegotiatedcharge",
                "charge_item_minnegotiatedcharge",
            ]
        ]
        .replace({",": ""}, regex=False)
        .apply(pd.to_numeric, 1)
    )
    # rename columns
    df.rename(columns=COLSDICT, inplace=True)

    # increment goes before everything else
    incremented = df["code"].copy()

    counter = defaultdict(int)
    for k, v in incremented.items():
        counter[v] += 1
        if counter[v] > 1:
            pre = v
            post = str("," + str(counter[v] - 1))
            incremented[k] = "%s%s" % (pre, post)
    # replace code column with new incremented version
    df["code"] = incremented

    # print("\n before procedures")
    # print(df.head(1))

    # create procedures table
    procedures = df[["code", "npi_number", "long_description"]].copy()
    procedures["short_description"] = np.nan
    # set columns in right order
    PROC_COLS = ["code", "npi_number", "short_description", "long_description"]

    procedures = procedures[PROC_COLS]

    # create procedures csv
    new_filename = "procedures\\" + lower_name + "_" + str(npi_number) + ".csv"
    procedures.to_csv(new_filename, index=False)

    # print("\n procedures")
    # print(procedures.head(1))

    # print("DEBUG")
    # CHARGES
    # select only payer-non specific charge info
    CHARGE_COLS = [
        "npi_number",
        "code",
        # "short_description",
        "self_pay",
        "gross",
        "max",
        "min",
    ]
    # create df to convert deidentified stuff to prices table
    charges = df[CHARGE_COLS].copy()
    # print("DEBUG 2")
    # rename columns
    charges.rename(columns=COLSDICT, inplace=True)

    # print("DEBUG 3")
    # melt charges
    def clean_melted_charges(dataframe):
        """used specifically to conver the metled non-payer specific charges to match prices schema"""

        # do the melting
        dataframe = dataframe.melt(id_vars=["npi_number", "code"], value_name="price")
        # set marker for inpatient & outpaitent prices
        dataframe["IP_OP"] = "UNSPECIFIED"
        dataframe["payer"] = "UNSPECIFIED"
        dataframe["caveat"] = np.nan
        dataframe["mode"] = np.NaN

        def replace_value(col, value_dict):
            for k, v in value_dict.items():
                dataframe.loc[dataframe.variable == k, col] = v

        # define melted variable name : variable new value
        payers = {
            "gross": "Gross Charge",
            "self_pay": "Discounted Cash Charge",
            "min": "De-identifiedMin",
            "max": "De-identifiedMax",
        }
        # replace payers with the above dict
        replace_value("payer", payers)

        # define melted variable name : variable new value
        modes = {
            "gross": "GROSS",
            "self_pay": "MINIMUM",
            "min": "MAXIMUM",
            "max": "EXPECTED",
        }
        # replace modes with the above dict
        replace_value("mode", modes)

        # return dataframe
        return dataframe[
            [
                "code",
                "npi_number",
                "payer",
                "IP_OP",
                "caveat",
                "mode",
                "price",
            ]
        ]

    # print("DEBUG 4")
    # call function
    new_charges = clean_melted_charges(charges)
    # print("DEBUG 5")
    # print("\n new charges")
    # print(new_charges.head(1))

    # END CHARGES
    # # flatten the xml tree structure to get payer and charge out to each row
    for i in range(2):
        # print(df.columns)
        df = df.pipe(flatten)
    df = col_clean(df)
    # rename columns
    df.rename(columns=COLSDICT, inplace=True)
    # print("\n df columns before df price")
    # print(df.head(1))

    # remove commas from prices and make numeric
    df["price"] = df["price"].replace({",": ""}, regex=False).apply(pd.to_numeric, 1)

    # drop unnecessary columns
    df.drop(
        columns=[
            # "short_description",
            "long_description",
            "charge_item_dept",
            "self_pay",
            "gross",
            "max",
            "min",
            "charge_item_note",
            "charge_item_qtytype",
            "charge_type",
            "createddate",
            "disclaimer",
        ],
        inplace=True,
    )

    # add nans as caveat
    df["IP_OP"] = "UNSPECIFIED"
    df["mode"] = "EXPECTED"
    df["caveat"] = "NONE"

    prices = pd.concat(
        [
            df[["code", "npi_number", "payer", "IP_OP", "caveat", "mode", "price"]],
            new_charges,
        ]
    )
    # create prices csv
    new_filename = "prices\\" + lower_name + "_" + str(npi_number) + ".csv"
    prices.to_csv(new_filename, index=False)

    # create payers csv
    new_filename = "payers\\" + lower_name + "_" + str(npi_number) + ".csv"
    payers = pd.DataFrame(prices.payer.unique())
    payers.to_csv(new_filename, index=False)

    print(
        "\n succesful: "
        + name
        + " - "
        + str(npi_number)
        + " price nrows: "
        + str(len(prices))
    )

# THIS WORKS mostly, but the xml items thing is a mess

In [11]:
for k in xmls.keys(): print(k)

1063713659
1801897038
1881726347
1780761866
1396760542
1831284280
1386720183
1497708473
1477874337
1699096552
1326113762
1881977593
1922319037
1245321181
1396842308
1154419737
1033160049
1134480155
1467763458
1811086994
1336231232


In [3]:
xmls={'1063713659': '27-3633811_CLINTON_MEMORIAL_HOSPITAL_StandardCharges.xml',
 '1801897038': '611739000_Conemaugh_Health_System_Standard_Charges_12282020.xml',
 '1881726347': '473937528_Fleming_County_Hospital_Standard_Charges_12202020.xml',
 '1780761866': '205220956 Havasu Regional Medical Center Standard Charges 12152020.xml',
 '1396760542': '621762419_Livingston_Regional_Hospital_Standard_Charges_12192020.xml',
 '1831284280': '36-4850536_LOURDES_HEALTH_StandardCharges.xml',
 '1386720183': '202028539_Sovah_Health_Martinsville_Standard_Charges_12182020.xml',
 '1497708473': '472546387_Nason_Hospital_Standard_Charges_12202020.xml',
 '1477874337': '27-2451336_NORTH_ALABAMA_MEDICAL_CENTER_StandardCharges.xml',
 '1699096552': '27-2451336_SHOALS_HOSPITAL_StandardCharges.xml',
 '1326113762': '742791525 Palestine Regional Medical Center Standard Charges 12162020.xml',
 '1881977593': '452909143 Person Memorial Standard Charges 12152020.xml',
 '1922319037': '621762469_Riverview_Regional_Medical_Center_Standard_Charges_12182020.xml',
 '1245321181': '300811171_Rutherford_Regional_Medical_Center_Standard_Charges_12202020.xml',
 '1396842308': '621762468_Sage_West_Standard_Charges_12192020.xml',
 '1154419737': '202028539_Sovah_Health_Danville_Standard_Charges_12182020.xml',
 '1033160049': '62-1771866_SAINT_MARYS_REGIONAL_HEALTH_SYSTEM_StandardCharges.xml',
 '1134480155': '621866028_Athens_Woods_Starr_Regional_Standard_Charges_12192020.xml',
 '1467763458': '272618876_Trousdale_Medical_Center_Standard_Charges_12192020.xml',
 '1811086994': '320063628_Valley_View_Medical_Center_Standard_Charges_12192020.xml',
 '1336231232': '611275266 Western Plains Medical Complex Standard Charges 12152020.xml'}
path='lifepoint_xmls'
for k, v in xmls.items():
    xmls[k]=path+"/"+xmls[k]    

xmls
successes = 0
total_prices = 0
for npi_number, raw_xml in failed_zips.items():
    # count loops
    df_raw = pdx.read_xml(
            pdx.read_xml_from_path(raw_xml, "utf8"),
            ["StandardCharges", "Facility"],
        )
    try:
        
        
        

        # create dictionary of xml names to column names
        COLSDICT = {
            # should end up as npi instead?
            "name": "name",
            # procedures
            "charge_item_code": "code",
            "charge_item_descr": "long_description",
            # de-identified prices, look at how we did this with BSMH
            "charge_item_discountcashcharge": "self_pay",
            "charge_item_grosscharge": "gross",
            "charge_item_maxnegotiatedcharge": "max",
            "charge_item_minnegotiatedcharge": "min",
            # payer negotiated prices
            "charge_item_contracts_contract_charge": "price",
            "charge_item_contracts_contract_payer": "payer",
        }

        df = df_raw.copy()
        # npi_number=1457321036
        # add column for npi number
        df["npi_number"] = npi_number

        def col_clean(data):
            """clean columsn names of pd dataframe"""

            data.columns = (
                data.columns.str.lower()
                .str.replace(" ", "_", regex=False)
                .str.replace("-", "_", regex=False)
                .str.replace("@", "", regex=False)
                .str.replace("|", "_", regex=False)
            )
            return data

        # flatten the xml tree structure to get code, description, and standard charges
        
        for i in range(5):
            # print(df.columns)
            df = df.pipe(flatten)
        
        # clean columns    
        df = col_clean(df)
        
        # TODO maybe we've done this wrong?
        df = df[df["charge_type"] == "HCPCS/CPT®"]
        df = df.dropna(subset=["charge_item_code"])
        
        try:
            # filter for relevant codes
            successes += 1
        except:
            failed_zips[npi_number]=xmls[npi_number]
            print("Charge|@Type ERROR: "+ str(raw_xml))
            continue
        # drop if no code
        
        try:
            # filter for relevant codes
        except:
            print("charge_item_code ERROR: "+ str(raw_xml))
            continue

        # define hospital name, for debugging
        name = df.name.max()
        lower_name = (
            df.name.str.lower()
            .str.replace(" ", "_", regex=False)
            .str.replace("-", "_", regex=False)
            .str.replace("@", "", regex=False)
            .str.replace("|", "_", regex=False)
            .max()
        )

        # remove commas from prices and make numeric
        df[
            [
                "charge_item_discountcashcharge",
                "charge_item_grosscharge",
                "charge_item_maxnegotiatedcharge",
                "charge_item_minnegotiatedcharge",
            ]
        ] = (
            df[
                [
                    "charge_item_discountcashcharge",
                    "charge_item_grosscharge",
                    "charge_item_maxnegotiatedcharge",
                    "charge_item_minnegotiatedcharge",
                ]
            ]
            .replace({",": ""}, regex=False)
            .apply(pd.to_numeric, 1)
        )
        # rename columns
        df.rename(columns=COLSDICT, inplace=True)

        # increment goes before everything else
        incremented = df["code"].copy()

        counter = defaultdict(int)
        for k, v in incremented.items():
            counter[v] += 1
            if counter[v] > 1:
                pre = v
                post = str("," + str(counter[v] - 1))
                incremented[k] = "%s%s" % (pre, post)
        # replace code column with new incremented version
        df["code"] = incremented

        # print("\n before procedures")
        # print(df.head(1))

        # create procedures table
        procedures = df[["code", "npi_number", "long_description"]].copy()
        procedures["short_description"] = np.nan
        
        # clean
        
        # set columns in right order
        PROC_COLS = ["code", "npi_number", "short_description", "long_description"]

        procedures = procedures[PROC_COLS]

        # create procedures csv
        new_filename = "procedures\\" + lower_name + "_" + str(npi_number) + ".csv"
        procedures.to_csv(new_filename, index=False)

        # print("\n procedures")
        # print(procedures.head(1))

        # print("DEBUG")
        # CHARGES
        # select only payer-non specific charge info
        CHARGE_COLS = [
            "npi_number",
            "code",
            # "short_description",
            "self_pay",
            "gross",
            "max",
            "min",
        ]
        # create df to convert deidentified stuff to prices table
        charges = df[CHARGE_COLS].copy()
        # print("DEBUG 2")
        # rename columns
        charges.rename(columns=COLSDICT, inplace=True)

        # print("DEBUG 3")
        # melt charges
        def clean_melted_charges(dataframe):
            """used specifically to conver the metled non-payer specific charges to match prices schema"""

            # do the melting
            dataframe = dataframe.melt(id_vars=["npi_number", "code"], value_name="price")
            # set marker for inpatient & outpaitent prices
            dataframe["IP_OP"] = "UNSPECIFIED"
            dataframe["payer"] = "UNSPECIFIED"
            dataframe["caveat"] = "NONE"
            dataframe["mode"] = np.NaN

            def replace_value(col, value_dict):
                for k, v in value_dict.items():
                    dataframe.loc[dataframe.variable == k, col] = v

            # define melted variable name : variable new value
            payers = {
                "gross": "Gross Charge",
                "self_pay": "Discounted Cash Charge",
                "min": "De-identifiedMin",
                "max": "De-identifiedMax",
            }
            # replace payers with the above dict
            replace_value("payer", payers)

            # define melted variable name : variable new value
            modes = {
                "gross": "GROSS",
                "self_pay": "MINIMUM",
                "min": "MAXIMUM",
                "max": "EXPECTED",
            }
            # replace modes with the above dict
            replace_value("mode", modes)

            # return dataframe
            return dataframe[
                [
                    "code",
                    "npi_number",
                    "payer",
                    "IP_OP",
                    "caveat",
                    "mode",
                    "price",
                ]
            ]

        # print("DEBUG 4")
        # call function
        new_charges = clean_melted_charges(charges)
        # print("DEBUG 5")
        # print("\n new charges")
        # print(new_charges.head(1))

        # END CHARGES
        # # flatten the xml tree structure to get payer and charge out to each row
        for i in range(2):
            # print(df.columns)
            df = df.pipe(flatten)
        df = col_clean(df)
        # rename columns
        df.rename(columns=COLSDICT, inplace=True)
        # print("\n df columns before df price")
        # print(df.head(1))

        # remove commas from prices and make numeric
        df["price"] = df["price"].replace({",": ""}, regex=False).apply(pd.to_numeric, 1)

        # drop unnecessary columns
        df.drop(
            columns=[
                # "short_description",
                "long_description",
                "charge_item_dept",
                "self_pay",
                "gross",
                "max",
                "min",
                "charge_item_note",
                "charge_item_qtytype",
                "charge_type",
                "createddate",
                "disclaimer",
            ],
            inplace=True,
        )

        # add nans as caveat
        df["IP_OP"] = "UNSPECIFIED"
        df["mode"] = "EXPECTED"
        df["caveat"] = "NONE"

        prices = pd.concat(
            [
                df[["code", "npi_number", "payer", "IP_OP", "caveat", "mode", "price"]],
                new_charges,
            ]
        )
        
        # make sure not nan prices
        prices=prices[prices.price.notnull()]
        
        # create prices csv
        new_filename = "prices\\" + lower_name + "_" + str(npi_number) + ".csv"
        prices.to_csv(new_filename, index=False)
        total_prices+=len(prices)

        # create payers csv
        new_filename = "payers\\" + lower_name + "_" + str(npi_number) + ".csv"
        payers = pd.DataFrame(prices.payer.unique())
        payers.to_csv(new_filename, index=False)

        print(
            "\n succesful: "
            + name
            + " - "
            + str(npi_number)
            + " price nrows: "
            + str(len(prices))
        )
    except:
        print(error fug)
        continue

print(successes)
print(total_prices)
failed_zips

Charge|@Type ERROR: lifepoint_xmls/27-3633811_CLINTON_MEMORIAL_HOSPITAL_StandardCharges.xml
Charge|@Type ERROR: lifepoint_xmls/36-4850536_LOURDES_HEALTH_StandardCharges.xml
Charge|@Type ERROR: lifepoint_xmls/27-2451336_NORTH_ALABAMA_MEDICAL_CENTER_StandardCharges.xml
Charge|@Type ERROR: lifepoint_xmls/27-2451336_SHOALS_HOSPITAL_StandardCharges.xml
Charge|@Type ERROR: lifepoint_xmls/62-1771866_SAINT_MARYS_REGIONAL_HEALTH_SYSTEM_StandardCharges.xml
16
0


{'1063713659': 'lifepoint_xmls/27-3633811_CLINTON_MEMORIAL_HOSPITAL_StandardCharges.xml',
 '1831284280': 'lifepoint_xmls/36-4850536_LOURDES_HEALTH_StandardCharges.xml',
 '1477874337': 'lifepoint_xmls/27-2451336_NORTH_ALABAMA_MEDICAL_CENTER_StandardCharges.xml',
 '1699096552': 'lifepoint_xmls/27-2451336_SHOALS_HOSPITAL_StandardCharges.xml',
 '1033160049': 'lifepoint_xmls/62-1771866_SAINT_MARYS_REGIONAL_HEALTH_SYSTEM_StandardCharges.xml'}

In [8]:
print(successes)
print(total_prices)
failed_zips

16
0


{'1063713659': 'lifepoint_xmls/27-3633811_CLINTON_MEMORIAL_HOSPITAL_StandardCharges.xml',
 '1831284280': 'lifepoint_xmls/36-4850536_LOURDES_HEALTH_StandardCharges.xml',
 '1477874337': 'lifepoint_xmls/27-2451336_NORTH_ALABAMA_MEDICAL_CENTER_StandardCharges.xml',
 '1699096552': 'lifepoint_xmls/27-2451336_SHOALS_HOSPITAL_StandardCharges.xml',
 '1033160049': 'lifepoint_xmls/62-1771866_SAINT_MARYS_REGIONAL_HEALTH_SYSTEM_StandardCharges.xml'}

In [ ]:
semi_failed_zips={'1063713659': 'lifepoint_xmls/27-3633811_CLINTON_MEMORIAL_HOSPITAL_StandardCharges.xml',
 '1831284280': 'lifepoint_xmls/36-4850536_LOURDES_HEALTH_StandardCharges.xml',
 '1477874337': 'lifepoint_xmls/27-2451336_NORTH_ALABAMA_MEDICAL_CENTER_StandardCharges.xml',
 '1699096552': 'lifepoint_xmls/27-2451336_SHOALS_HOSPITAL_StandardCharges.xml',
 '1033160049': 'lifepoint_xmls/62-1771866_SAINT_MARYS_REGIONAL_HEALTH_SYSTEM_StandardCharges.xml'}

In [6]:
import sys
sys.getsizeof(df)/1e6

126.322509

In [8]:
charges.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1672 entries, 0 to 1925
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   name                             1672 non-null   object 
 1   charge_item_code                 1672 non-null   object 
 2   charge_item_descr                1672 non-null   object 
 3   charge_item_discountcashcharge   1672 non-null   float64
 4   charge_item_grosscharge          1672 non-null   float64
 5   charge_item_maxnegotiatedcharge  1613 non-null   float64
 6   charge_item_minnegotiatedcharge  1613 non-null   float64
dtypes: float64(4), object(3)
memory usage: 104.5+ KB


In [39]:
!pip install openpyxl
# batch_1=pd.read_csv(r"lifepoint_batch_1.csv")
filename = 'lifepoint_free_hosp.xlsx'
sheet='lifepoint_free_hosp'
batch_1=pd.read_excel(filename,sheet)
batch_1=batch_1.loc[batch_1.batch==1]
# for col in batch_1.columns:
#     print("'"+col+"':,")
    
# clean data
batch_1.rename(columns={'provider':'name','npi':'npi_number'},inplace=True)
batch_1['publish_date']=np.nan
batch_1.npi_number=batch_1.npi_number.astype('str')
ORDERED_COLS=[
    'npi_number',
    'name',
    'url',
    'street_address',
    'city',
    'state',
    'zip_code',
    'publish_date',
]
batch_1=batch_1[ORDERED_COLS]

#export to csv
batch_1.to_csv(r'hospitals/batch_1.csv', index=False)

'Unnamed: 0':,
'provider':,
'npi_number':,
'zip_code':,
'street_address':,
'city':,
'county':,
'state':,
'acute_care_facility':,
'hospital_type':,
'compliance_score':,
'in_procedures':,
'in_prices':,
'batch':,
'taken':,
'link':,
'url':,


In [23]:
print('[')
for item in batch_1.columns.to_list():
    print("    '"+str(item)+"',")
print(']')

[
    'name',
    'npi_number',
    'zip_code',
    'street_address',
    'city',
    'county',
    'state',
    'url',
    'publish_date',
]


In [ ]:
# null_thing=" . ($publish_date==NULL ? "NULL" : "'$publish_date'") . "

# !dolt sql -r -csv -q "SELECT npi_number FROM hospitals" > hospital_all_npis.csv


In [31]:

xmls={'1063713659': '27-3633811_CLINTON_MEMORIAL_HOSPITAL_StandardCharges.xml',
 '1801897038': '611739000_Conemaugh_Health_System_Standard_Charges_12282020.xml',
 '1881726347': '473937528_Fleming_County_Hospital_Standard_Charges_12202020.xml',
 '1780761866': '205220956 Havasu Regional Medical Center Standard Charges 12152020.xml',
 '1396760542': '621762419_Livingston_Regional_Hospital_Standard_Charges_12192020.xml',
 '1831284280': '36-4850536_LOURDES_HEALTH_StandardCharges.xml',
 '1386720183': '202028539_Sovah_Health_Martinsville_Standard_Charges_12182020.xml',
 '1497708473': '472546387_Nason_Hospital_Standard_Charges_12202020.xml',
 '1477874337': '27-2451336_NORTH_ALABAMA_MEDICAL_CENTER_StandardCharges.xml',
 '1699096552': '27-2451336_SHOALS_HOSPITAL_StandardCharges.xml',
 '1326113762': '742791525 Palestine Regional Medical Center Standard Charges 12162020.xml',
 '1881977593': '452909143 Person Memorial Standard Charges 12152020.xml',
 '1922319037': '621762469_Riverview_Regional_Medical_Center_Standard_Charges_12182020.xml',
 '1245321181': '300811171_Rutherford_Regional_Medical_Center_Standard_Charges_12202020.xml',
 '1396842308': '621762468_Sage_West_Standard_Charges_12192020.xml',
 '1154419737': '202028539_Sovah_Health_Danville_Standard_Charges_12182020.xml',
 '1033160049': '62-1771866_SAINT_MARYS_REGIONAL_HEALTH_SYSTEM_StandardCharges.xml',
 '1134480155': '621866028_Athens_Woods_Starr_Regional_Standard_Charges_12192020.xml',
 '1467763458': '272618876_Trousdale_Medical_Center_Standard_Charges_12192020.xml',
 '1811086994': '320063628_Valley_View_Medical_Center_Standard_Charges_12192020.xml',
 '1336231232': '611275266 Western Plains Medical Complex Standard Charges 12152020.xml'}

batch1=pd.read_csv('lifepoint_batch_1.csv')
path='lifepoint_xmls'
for k, v in xmls.items():
    xmls[k]=path+"/"+xmls[k]


batch1_successes=batch1.loc[batch1.npi.astype('str').isin(xmls.keys()),['provider','npi']]

# for row in batch1_successes:
#     print(row)

for k in xmls.keys():
    print(tuple(k))

tuple(xmls.keys()Import 15 Lifepoint hospitals, prices, payers, procedures)

('1', '0', '6', '3', '7', '1', '3', '6', '5', '9')
('1', '8', '0', '1', '8', '9', '7', '0', '3', '8')
('1', '8', '8', '1', '7', '2', '6', '3', '4', '7')
('1', '7', '8', '0', '7', '6', '1', '8', '6', '6')
('1', '3', '9', '6', '7', '6', '0', '5', '4', '2')
('1', '8', '3', '1', '2', '8', '4', '2', '8', '0')
('1', '3', '8', '6', '7', '2', '0', '1', '8', '3')
('1', '4', '9', '7', '7', '0', '8', '4', '7', '3')
('1', '4', '7', '7', '8', '7', '4', '3', '3', '7')
('1', '6', '9', '9', '0', '9', '6', '5', '5', '2')
('1', '3', '2', '6', '1', '1', '3', '7', '6', '2')
('1', '8', '8', '1', '9', '7', '7', '5', '9', '3')
('1', '9', '2', '2', '3', '1', '9', '0', '3', '7')
('1', '2', '4', '5', '3', '2', '1', '1', '8', '1')
('1', '3', '9', '6', '8', '4', '2', '3', '0', '8')
('1', '1', '5', '4', '4', '1', '9', '7', '3', '7')
('1', '0', '3', '3', '1', '6', '0', '0', '4', '9')
('1', '1', '3', '4', '4', '8', '0', '1', '5', '5')
('1', '4', '6', '7', '7', '6', '3', '4', '5', '8')
('1', '8', '1', '1', '0', '8', 

('1063713659',
 '1801897038',
 '1881726347',
 '1780761866',
 '1396760542',
 '1831284280',
 '1386720183',
 '1497708473',
 '1477874337',
 '1699096552',
 '1326113762',
 '1881977593',
 '1922319037',
 '1245321181',
 '1396842308',
 '1154419737',
 '1033160049',
 '1134480155',
 '1467763458',
 '1811086994',
 '1336231232')

15 Lifepoint hospitals - import hospitals, prices, payers, procedures

('1063713659',
 '1801897038',
 '1881726347',
 '1780761866',
 '1396760542',
 '1831284280',
 '1386720183',
 '1497708473',
 '1477874337',
 '1699096552',
 '1326113762',
 '1881977593',
 '1922319037',
 '1245321181',
 '1396842308',
 '1154419737',
 '1033160049',
 '1134480155',
 '1467763458',
 '1811086994',
 '1336231232')